In [1]:
import sys
sys.path.append('./codes/forgraph/')
from config import args
from sklearn.metrics import roc_auc_score
from models import GCN
from metrics import *
import numpy as np
from Explainer import Explainer
from scipy.sparse import coo_matrix,csr_matrix
import networkx as nx
skip = 5
topk = 5
import matplotlib
import matplotlib.pyplot as plt
import pickle as pkl


args.elr = 0.003
args.coff_t0=5.0
args.coff_t0=1.0
args.coff_size = 0.00
args.coff_ent = 0.000



In [2]:
with open('./dataset/BA-2motif.pkl','rb') as fin:
    adjs,features,labels = pkl.load(fin)

model = GCN(input_dim=features.shape[-1], output_dim=labels.shape[1])
model.load_weights(args.save_path+args.dataset)


embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
                               tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
                            tf.convert_to_tensor(adjs,dtype=tf.float32)),training=False)
pred_label = tf.argmax(outputs, 1)

acc = accuracy(outputs, labels)
print('model acc',acc.numpy())

embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
                               tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
                            tf.convert_to_tensor(adjs,dtype=tf.float32)),training=False)
pred_label = tf.argmax(outputs, 1)

model acc 1.0


In [3]:
print(adjs, features, labels)

[[[0. 1. 0. ... 0. 0. 0.]
  [1. 0. 1. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 1.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[0. 1. 1. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 1.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [1. 0. 1. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 1.]
  [0. 0. 0. ... 0. 1. 0.]]

 ...

 [[0. 1. 1. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [1. 0. 1. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [1. 0. 1. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]] [[[0.1 0.1

In [ ]:
if args.setting==1:
    allnodes = [i for i in range(0,100)]
elif args.setting==2:
    allnodes = [i for i in range(0,100)]
    allnodes.extend([i for i in range(500,600)])
elif args.setting==3:
    allnodes=[i for i in range(1000)]
explainer = Explainer(model=model,nodesize=adjs.shape[1])
optimizer = tf.keras.optimizers.Adam(learning_rate=args.elr)


In [ ]:
def acc(adj,insert):
    mask = explainer.masked_adj.numpy()
    adj = coo_matrix(adj)
    for r,c in list(zip(adj.row,adj.col)):
        if r>=insert and r<insert+skip and c>=insert and c<insert+skip:
            reals.append(1)
        else:
            reals.append(0)
        preds.append(mask[r][c])

In [ ]:
def explain_graph(gid):
    fea,emb,adj,label,graphid = features[gid], embs[gid], adjs[gid], labels[gid], gid
    explainer((fea,emb,adj,1.0,label))
    insert = 20
    acc(adj,insert)

In [ ]:
def test():
    global preds
    global reals
    preds = []
    reals = []
    for gid in allnodes:
        explain_graph(gid)
    auc = roc_auc_score(reals,preds)
    return auc

In [ ]:
def train():
    epochs = 10
    t0 = args.coff_t0
    t1 = args.coff_te

    for epoch in range(epochs): 
        loss = 0
        tmp = float(t0 * np.power(t1 / t0, epoch /epochs))
        train_instances = [ins for ins in range(adjs.shape[0])]
        np.random.shuffle(train_instances)
        for gid in train_instances:
            with tf.GradientTape() as tape:
                pred = explainer((features[gid],embs[gid],adjs[gid],tmp, labels[gid]),training=True)
                loss += explainer.loss(pred, labels[gid])
        train_variables = [para for para in explainer.trainable_variables]
        grads = tape.gradient(loss, train_variables)
        optimizer.apply_gradients(zip(grads, train_variables))

In [ ]:
auc_best = -np.inf
best_k = None
k = 20
for i in range(0, k):
    train()
    auc = test()
    print(i)
    if auc > auc_best:
        auc_best = auc
        best_k = k

In [ ]:
print(auc_best)
print(best_k)